In [1]:
%pip install transformers datasets
%pip install scikit-learn
%pip install torch
%pip install --upgrade transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
# Import necessary libraries
import pandas as pd
# from google.colab import files

# Just manually provide the CSV file path
file_path = 'Soil_merged.csv'  # replace with your actual file name or path
data = pd.read_csv(file_path)


# Display the first few rows
data.head()

,fid,Survey_No,STATE,DISTRICT,TEHSIL,pH,EC,OC,n,p,...,field_id,polarization_diff_s1,ndvi_s2,bsi_s2,osavi_s2,gndvi_s2,sr1_s2,sr2_s2,ndwi_s2,mean_nir_band_s2
0,1,11-Jan,Gujrat,NaN,NaN,7.91,0.22,0.42,NaN,10.0,...,284,17.300871,0.142846,0.205670,0.090762,0.322064,1.334322,0.720174,-0.322064,0.162560
1,2,11-Feb,Gujrat,NaN,NaN,8.09,0.19,0.60,NaN,6.0,...,283,21.555414,0.145391,0.192945,0.093909,0.346883,1.347734,0.675383,-0.346883,0.168674
2,3,140,Gujrat,NaN,NaN,8.22,0.30,0.54,NaN,6.0,...,310,22.132469,0.127440,0.202547,0.085367,0.328108,1.294065,0.671539,-0.328108,0.189712
3,4,22,Gujrat,NaN,NaN,7.46,0.56,0.54,NaN,10.0,...,272,18.114547,0.151544,0.192670,0.098100,0.327093,1.360491,0.743541,-0.327093,0.173924
4,5,48,Gujrat,NaN,NaN,7.74,0.31,0.52,NaN,14.0,...,301,10.778053,0.149600,0.188388,0.103821,0.299159,1.353599,0.747119,-0.299159,0.217503


In [3]:

# Select relevant satellite features
satellite_features = [
    'polarization_diff_s1', 'ndvi_s2', 'bsi_s2', 'osavi_s2',
    'gndvi_s2', 'sr1_s2', 'sr2_s2', 'ndwi_s2', 'mean_nir_band_s2'
]

# Create input string for GPT
data['input_string'] = data[satellite_features].apply(lambda row: ', '.join(
    [f"{col}: {row[col]:.4f}" for col in satellite_features]), axis=1)
data['input_string'] = data['input_string'] + ' => P: ' + data['p'].astype(str)


# Display a few examples of the input strings
data[['input_string']].head()


,input_string
0,"polarization_diff_s1: 17.3009, ndvi_s2: 0.1428..."
1,"polarization_diff_s1: 21.5554, ndvi_s2: 0.1454..."
2,"polarization_diff_s1: 22.1325, ndvi_s2: 0.1274..."
3,"polarization_diff_s1: 18.1145, ndvi_s2: 0.1515..."
4,"polarization_diff_s1: 10.7781, ndvi_s2: 0.1496..."


In [4]:

# STEP 2: Split dataset
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(data[['input_string', 'p']], test_size=0.2, random_state=42)

# STEP 3: Convert to Hugging Face dataset
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [5]:


# STEP 4: Tokenization
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

def tokenize_function(examples):
    tokenized = tokenizer(examples['input_string'], padding="max_length", truncation=True, max_length=128)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [ ]:
import transformers
import accelerate
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)

Transformers: 4.51.3
Accelerate: 1.6.0


In [ ]:
# STEP 5: Model Training
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
import transformers
import accelerate

model = GPT2LMHeadModel.from_pretrained("distilgpt2")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # 👈 This disables W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


In [ ]:
# STEP 6: Save the trained model
save_directory = './trained_model'
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Model and tokenizer saved to {save_directory}")

In [ ]:
# STEP 7: Evaluate model using numeric regression metrics
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Generate predictions on eval data
model.eval()
predicted_ps = []
true_ps = []

for row in eval_df['input_string']:
    input_ids = tokenizer(row, return_tensors="pt", truncation=True, padding=True).input_ids
    with torch.no_grad():
        output = model.generate(input_ids, max_length=128, num_return_sequences=1)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract predicted P value from decoded output
    try:
        pred_value = float(decoded.split('=> P:')[-1].strip().split()[0])
        predicted_ps.append(pred_value)
    except:
        predicted_ps.append(np.nan)  # Handle decoding issues

# Clean up nan predictions
eval_df_clean = eval_df.copy()
eval_df_clean['predicted_p'] = predicted_ps
eval_df_clean = eval_df_clean.dropna()

# Calculate regression metrics
y_true = eval_df_clean['p'].astype(float)
y_pred = eval_df_clean['predicted_p']

mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"\n📊 Evaluation Metrics on Test Set:")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")